In [26]:
# Demo for synchronization of two data directories
import os
import io
import pandas as pd
import numpy as np
import farmhash
import time
import sqlite3

In [4]:
def create_dirdf(directory):
    if not os.path.exists(directory):
        print("Error: Directory '" + directory + "' does not exist.")
        return
    
    filenames = []
    hashvalues = []
    
    for root, subdir, files in os.walk(directory):
        for name in files:
            if not name[0] == ".": # ignore hidden files
                filepath = os.path.join(root, name)
                
                # hash full file contents
                # note: spice data encoding is mixed, so read as binary
                file = str(io.open(filepath,'rb').read()) 
                filenames.append(filepath.split(directory, 1)[1])
                hashvalues.append(farmhash.hash64(file))
                
    df = pd.DataFrame(data=hashvalues, index = filenames, columns = ["Hash"])
    df.index.name = directory
    return df

start = time.time()
dir1df = create_dirdf("./testdir1")
print("elapsed time: ", time.time() - start)

dir2df = create_dirdf("./testdir2")

print(dir1df, dir2df)

elapsed time:  0.0032210350036621094
                                           Hash
./testdir1                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950                                            Hash
./testdir2                                     
/testfile1.txt             13167233149662072294
/testfile2.txt              2116770068367243914
/testfile3.txt             10117441339441774812
/testfile4.txt               407662078023551858
/testdir1A/testfile1A.txt   1855841718642996950


In [138]:
os.remove('./spicedb.sqlite')

In [139]:
def create_spicedb(spdir):
    if not os.path.exists(spdir):
        print("Error: Directory '" + spdir + "' does not exist.")
        return
    
    conn = sqlite3.connect('./spicedb.sqlite') # initialize db, this might move to an init func in the api
    c = conn.cursor()
    
    c.execute("CREATE TABLE SPICE (Mission TEXT)") # create missions table
    c.execute("ALTER TABLE SPICE ADD COLUMN Kernel TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN File TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN Path TEXT")
    c.execute("ALTER TABLE SPICE ADD COLUMN Hash TEXT") #theres probs a way to to this in one line


    # ooh spicy ~
    for mis in [m for m in os.listdir(spdir) if not m[0] == '.']:
        for ker in [k for k in os.listdir(spdir+'/'+mis) if not k[0] == '.']:
            
            # first two directory layers should always be mission and kernels - then it gets mixed
            for root, subdir, files in os.walk(spdir+'/'+mis+'/'+ker):
                for name in files:
                    if not name[0] == ".": # ignore hidden files
                        # more tabs pls
                        filepath = os.path.join(root, name)
                        mis_hr = missions_readable[mis]

                        # spice data encoding is mixed, so read as binary
                        fhash = farmhash.hash64(str(io.open(filepath,'rb').read()))
                        c.execute("INSERT OR IGNORE INTO SPICE (Mission, Kernel, File, Path, Hash) VALUES ('{mn}', '{kn}', '{fn}', '{fp}', '{fh}')"\
                                  .format(mn=mis_hr, kn=ker, fn=name, fp=filepath, fh=fhash))
    
    conn.commit()

create_spicedb('./spice_data')

In [132]:
conn = sqlite3.connect('./spicedb.sqlite')
c = conn.cursor()
c.execute("SELECT * FROM SPICE WHERE Mission= 'clem1-l-spice-6-v1.0'")
# Note: fetchall() will pull the whole buffer, if you SELECT ten times, the result will be in there ten times
all_rows = c.fetchall() 
print(all_rows)

[('clem1-l-spice-6-v1.0', 'ck', 'testfile1.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile1.txt', '13167233149662072294'), ('clem1-l-spice-6-v1.0', 'ck', 'testfile2.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile2.txt', '2116770068367243914'), ('clem1-l-spice-6-v1.0', 'ck', 'testfile3.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile3.txt', '10117441339441774812'), ('clem1-l-spice-6-v1.0', 'ck', 'testfile4.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testfile4.txt', '407662078023551858'), ('clem1-l-spice-6-v1.0', 'ck', 'testfile1A.txt', './spice_data/clem1-l-spice-6-v1.0/ck/testdir1A/testfile1A.txt', '1855841718642996950')]


In [108]:
conn.close()

In [133]:
missions_readable = {   "clem1-l-spice-6-v1.0"       : "clementine",
                        "co-s_j_e_v-spice-6-v1.0"    : "cassini_orbiter",
                        "dawn-m_a-spice-6-v1.0"      : "dawn",
                        "di-c-spice-6-v1.0"          : "deep_impact",
                        "dif-c_e_x-spice-6-v1.0"     : "epoxi",
                        "ds1-a_c-spice-6-v1.0"       : "deep_space_1",
                        "grail-l-spice-6-v1.0"       : "grail",
                        "hay-a-spice-6-v1.0"         : "hayabusa",
                        "jno-j_e_ss-spice-6-v1.0"    : "juno",
                        "lro-l-spice-6-v1.0"         : "lunar_reconnaissance_orbiter",
                        "mer1-m-spice-6-v1.0"        : "mer_1",
                        "mer2-m-spice-6-v1.0"        : "mer_2",
                        "mess-e_v_h-spice-6-v1.0"    : "messenger",
                        "mex-e_m-spice-6-v1.0"       : "mars_express",
                        "mgs-m-spice-6-v1.0"         : "mars_global_surveyor",
                        "mro-m-spice-6-v1.0"         : "mars_reconnaissance_orbiter",
                        "msl-m-spice-6-v1.0"         : "mars_science_laboratory",
                        "near-a-spice-6-v1.0"        : "near",
                        "nh-j_p_ss-spice-6-v1.0"     : "new_horizons",
                        "ody-m-spice-6-v1.0"         : "mars_odyssey",
                        "ros-e_m_a_c-spice-6-v1.0"   : "rosetta",
                        "sdu-c-spice-6-v1.0"         : "stardust",
                        "vco-v-spice-6-v1.0"         : "venus_climate_orbiter",
                        "vex-e_v-spice-6-v1.0"       : "venus_express",
                        "vo1_vo2-m-spice-6-v1.0"     : "viking_orbiter"}

missions_true = {value: key for key, value in missions_readable.items()}